# Usar API para Localizar endereços completos

In [1]:
#pacotes
import pandas as pd
import urllib.request
import json

In [2]:
#importação, transforma em str para a API ler
df_original = pd.read_excel("C:\Python\dados.xlsx", dtype={'CEP':'str'})

# Análise Exploratória

In [3]:
df_original.shape

(999, 1)

In [4]:
df_original.head

<bound method NDFrame.head of           CEP
0    69043690
1    08130230
2    01404000
3    22221971
4    49230000
..        ...
994  21920240
995  08615070
996  88080760
997  01323050
998  04011060

[999 rows x 1 columns]>

In [5]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CEP     999 non-null    object
dtypes: object(1)
memory usage: 7.9+ KB


In [6]:
#remover duplicados
df_ceps_clientes = df_original['CEP']
df_ceps_clientes = pd.DataFrame(df_ceps_clientes)
df_ceps_clientes.drop_duplicates(inplace = True)
df_ceps_clientes.head()

,CEP
0,69043690
1,08130230
2,01404000
3,22221971
4,49230000


In [7]:
#qtde ceps únicos
print("Ceps únicos de clientes:", df_ceps_clientes.count())

Ceps únicos de clientes: CEP    971
dtype: int64


In [8]:
#consultar API, abaixo caminho, com um cep de exemplo 74491628 pra testar se funciona
url = 'https://viacep.com.br/ws/%s/json' % '74491628'
headers = {'User-Agent': 'Autociencia/1.0'}
#faz a chamada da API
requisicao = urllib.request.Request(url, headers=headers, method='GET')
cliente = urllib.request.urlopen(requisicao)
conteudo = cliente.read().decode('utf-8')
endereco = json.loads(conteudo)

In [9]:
#endereco completo, mas sem o tratamento dos dados
print(endereco)

{'cep': '74491-628', 'logradouro': 'Rua JC311', 'complemento': '', 'unidade': '', 'bairro': 'Residencial Jardins do Cerrado 7', 'localidade': 'Goiânia', 'uf': 'GO', 'ibge': '5208707', 'gia': '', 'ddd': '62', 'siafi': '9373'}


In [10]:
#separando os valores do dicionário
print('Endereco: ',endereco['logradouro'])
print('Bairro: ',endereco['bairro'])
print('Cidade: ',endereco['localidade'])
print('Estado: ',endereco['uf'])

Endereco:  Rua JC311
Bairro:  Residencial Jardins do Cerrado 7
Cidade:  Goiânia
Estado:  GO


# Todos os endereços

In [12]:
#iteraçao linha por linha no df
for idx, row in df_ceps_clientes.iterrows():
    #var cep pega a col cep
    cep = row.CEP
    url = 'https://viacep.com.br/ws/%s/json' % cep #coloca a var novamente
    headers = {'User-Agent': 'Autociencia/1.0'}
    #faz a chamada da API
    requisicao = urllib.request.Request(url, headers=headers, method='GET')
    cliente = urllib.request.urlopen(requisicao)
    conteudo = cliente.read().decode('utf-8')
    endereco = json.loads(conteudo)
    cliente.close() #deve fechar a chamada pq senao dá erro
    print(cep)
    #cria as colunas e popula com as informações
    if "cep" in endereco:
        df_ceps_clientes.at[idx, 'Endereco_cliente'] = endereco['logradouro']
        df_ceps_clientes.at[idx, 'Bairro_cliente'] = endereco['bairro']
        df_ceps_clientes.at[idx, 'Cidade_cliente'] = endereco['localidade']
        df_ceps_clientes.at[idx, 'Estado_cliente'] = endereco['uf']
        
    else:
        df_ceps_clientes.at[idx, 'Endereco_cliente'] = 'Não Localizado'
        df_ceps_clientes.at[idx, 'Bairro_cliente'] = 'Não Localizado'
        df_ceps_clientes.at[idx, 'Cidade_cliente'] = 'Não Localizado'
        df_ceps_clientes.at[idx, 'Estado_cliente'] = 'Não Localizado'
        

69043690
08130230
01404000
22221971
49230000
38057050
21321310
72153522
02736070
30220400
24020040
08715460
39801022
24130465
45028190
11030320
06407000
90620000
31741550
14890038
22240004
77760000
60170041
31730690
38406586
04104001
36088120
02801000
04930300
04131130
06663450
13295000
09061660
20560010
74455562
40391320
29164063
12951110
72010070
01550060
62640000
11030906
73020035
08253030
52070160
01333030
01219011
09250000
35960000
71510260
31010260
22205040
09911310
96015000
85812002
24350360
39900000
07845130
07050001
04018001
25550202
25060040
42833000
22220001
12460000
13015301
24320570
04346000
20241110
21070170
38408118
38055420
22270000
13085200
28930000
22430200
04021070
86020040
08717620
70790140
21340090
20785330
05187430
31573450
06773350
49020190
57039710
06320270
02208000
72583400
90690190
36050025
83405250
22750012
71060636
05416000
11045200
09770340
22060020
22461230
22790671
36047040
22240006
60192450
20551100
90420030
22041011
91760710
90050102
20550050
01238904
4

URLError: <urlopen error [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu
corretamente após um período de tempo ou a conexão estabelecida falhou
porque o host conectado não respondeu>

In [13]:
df_ceps_clientes.head(30)

,CEP,Endereco_cliente,Bairro_cliente,Cidade_cliente,Estado_cliente
0,69043690,Rua Violeta Areosa,Alvorada,Manaus,AM
1,08130230,Rua Marco Antônio Setti,Cidade Kemel,São Paulo,SP
2,01404000,Alameda Campinas,Jardim Paulista,São Paulo,SP
3,22221971,Largo do Machado,Catete,Rio de Janeiro,RJ
4,49230000,,,Santa Luzia do Itanhy,SE
5,38057050,Alameda Vermelha,Grande Horizonte,Uberaba,MG
6,21321310,Rua Urucuia,Vila Valqueire,Rio de Janeiro,RJ
7,72153522,Setor SIG Área Especial 2,Taguatinga Norte (Taguatinga),Brasília,DF
8,02736070,Rua Miguel Helou,Parque Monteiro Soares,São Paulo,SP
9,30220400,Rua Capivari,Serra,Belo Horizonte,MG


In [14]:
df_ceps_clientes.to_csv('ceps_clientes.csv', index=False)

# Fim